# Import libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jadha\AppData\Roaming\nltk_data...


True

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jadha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# Read data

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [3]:
type(intents)

dict

In [4]:
intents['intents'][0]

{'tag': 'greeting',
 'patterns': ['Hi there',
  'How are you',
  'Is anyone there?',
  'Hey',
  'Hola',
  'Hello',
  'Good day'],
 'responses': ['Hello, thanks for asking',
  'Good to see you again',
  'Hi there, how can I help?'],
 'context': ['']}

In [5]:
nltk.word_tokenize('Is anyone there?')

['Is', 'anyone', 'there', '?']

In [6]:
intents['intents'][0]['patterns']

['Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day']

# Tokenization

In [7]:
 for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag']) 

In [8]:
#total no of words
len(words)

194

In [9]:
#unique words
words1=sorted(list(set(words)))
len(words1)

101

In [10]:
len(documents)

47

In [11]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

# Lammatization

In [12]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jadha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
lemmatizer = WordNetLemmatizer()

In [14]:
lemmatizer.lemmatize('runs')

'run'

In [15]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
len(words)

88

In [16]:
# sort classes
classes = sorted(list(set(classes)))
classes

['adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'goodbye',
 'greeting',
 'hospital_search',
 'options',
 'pharmacy_search',
 'thanks']

In [17]:
# documents = combination of patterns and intents
print (len(documents), "documents")

47 documents


In [18]:
# classes = intents
print (len(classes),"classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [19]:
# words ----> all words, vocabulary
print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [20]:
pickle.dump(words,open('words.pkl','wb'))

# Preparing fratures and  classes

In [21]:
# create an empty array for our output
output_empty=[0]*len(classes)
output_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [22]:
training = []
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
# random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\jadha\AppData\Local\Temp\ipykernel_35028\4160180180.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [23]:
np.array(train_x[34])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
train_x[34].count(1)

5

In [25]:
[w.lower() for w in documents[34][0] if w.lower() in words]

['load', 'patient', 'blood', 'pressure', 'result']

In [26]:
training

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 1, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
        list([0, 0, 0, 0, 1, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 1, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0,

In [27]:
train_y

[[0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1,

# Model

Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax

In [28]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11392     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 20,233
Trainable params: 20,233
Non-trainable params: 0
_________________________________________________________________


 Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [29]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\jadha\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [30]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("hey model is created")

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.2455 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1343 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0390 - accuracy: 0.2340
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9081 - accuracy: 0.3617
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8706 - accuracy: 0.3830
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8000 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7262 - accuracy: 0.4043
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5957 - accuracy: 0.4681
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3303 - accuracy: 0.6596
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.1384 - accuracy: 0.7234
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.0607 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0619 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0881 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0693 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0786 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0268 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0529 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0511 - accuracy: 0.9574
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0163 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0366 - accuracy: 0.9787
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0276 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 1.0000
Epoc

In [31]:
model.evaluate(train_x,train_y)

2/2 [==============================] - 4s 2ms/step - loss: 3.6921e-05 - accuracy: 1.0000


[3.6921272112522274e-05, 1.0]

In [33]:
y_predict=[classes[np.argmax(i)] for i in model.predict(train_x)]
y_predict

2/2 [==============================] - 0s 3ms/step


['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'options',
 'options',
 'options',
 'options',
 'options',
 'adverse_drug',
 'adverse_drug',
 'adverse_drug',
 'adverse_drug',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure',
 'blood_pressure',
 'blood_pressure',
 'blood_pressure',
 'blood_pressure_search',
 'blood_pressure_search',
 'blood_pressure_search',
 'blood_pressure_search',
 'blood_pressure_search',
 'pharmacy_search',
 'pharmacy_search',
 'pharmacy_search',
 'pharmacy_search',
 'pharmacy_search',
 'hospital_search',
 'hospital_search',
 'hospital_search',
 'hospital_search',
 'hospital_search']

In [35]:
#actual values
y_actual=[classes[np.argmax(i)] for i in train_y]
np.array(y_actual)

array(['greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye',
       'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks',
       'options', 'options', 'options', 'options', 'options',
       'adverse_drug', 'adverse_drug', 'adverse_drug', 'adverse_drug',
       'adverse_drug', 'blood_pressure', 'blood_pressure',
       'blood_pressure', 'blood_pressure', 'blood_pressure',
       'blood_pressure_search', 'blood_pressure_search',
       'blood_pressure_search', 'blood_pressure_search',
       'blood_pressure_search', 'pharmacy_search', 'pharmacy_search',
       'pharmacy_search', 'pharmacy_search', 'pharmacy_search',
       'hospital_search', 'hospital_search', 'hospital_search',
       'hospital_search', 'hospital_search'], dtype='<U21')

In [38]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [39]:
confusion_matrix(y_actual,y_predict)

array([[5, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 5, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 5, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 5, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 7, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 5, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 5]], dtype=int64)

In [40]:
accuracy_score(y_actual,y_predict)

1.0

# Catbot Creation

In [41]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))

## clean the sentences

In [42]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [43]:
clean_up_sentence(' HI ANTONE IS THERE')

['hi', 'antone', 'is', 'there']

## Bag of word 

return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [44]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [46]:
bow('HI ANTONE IS THERE',words,show_details=True)

found in bag: hi
found in bag: is
found in bag: there


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [50]:
y_predict=model.predict(bow('HI ANTONE IS THERE',words,show_details=False).reshape(1,88))
y_predict

1/1 [==============================] - 0s 27ms/step


array([[3.82893255e-08, 1.03835305e-07, 3.47662095e-08, 3.78771603e-07,
        9.99997854e-01, 5.39436201e-07, 5.71281589e-07, 5.42647626e-07,
        2.96076799e-08]], dtype=float32)

In [52]:
classes[[np.argmax(i) for i in y_predict][0]]

'greeting'

## prediction

In [53]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [54]:
predict_class('thanks kishor',model)

1/1 [==============================] - 0s 32ms/step


[{'intent': 'thanks', 'probability': '0.99995565'}]

# Responce

In [56]:
intents['intents'][0]['responses']

['Hello, thanks for asking',
 'Good to see you again',
 'Hi there, how can I help?']

In [57]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [58]:
chatbot_response('hi')

1/1 [==============================] - 0s 31ms/step


'Hi there, how can I help?'

In [59]:
chatbot_response('i am not felling so well')

1/1 [==============================] - 0s 35ms/step


'Hi there, how can I help?'

In [60]:
chatbot_response('I want to search for blood pressure result history')

1/1 [==============================] - 0s 32ms/step


'Please provide Patient ID'

# GUI

In [61]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


1/1 [==============================] - 0s 23ms/step
